In [ ]:
using DrWatson
@quickactivate "masterarbeit"

In [ ]:
using BenchmarkTools
using ProgressMeter
using CUDA
using Flux
using LaTeXStrings
using Flux: train!
using GLMakie
using Printf
using Dates
using JLD2
using TOML
using StatsBase # for fit(histogram)

In [ ]:
Makie.inline!(true)
fontsize_theme = Theme(fontsize=35)
set_theme!(fontsize_theme)
wblue = Makie.wong_colors()[1]
worange = Makie.wong_colors()[2]
wgreen = Makie.wong_colors()[3]
wpink = Makie.wong_colors()[4]
wlblue = Makie.wong_colors()[5]
worange = Makie.wong_colors()[6]
wyellow = Makie.wong_colors()[7];

In [ ]:
using Revise

In [ ]:
using masterarbeit

In [ ]:
function lossf(m::Chain, cm::ChannelMapping, f::Function, x::T) where T<:AbstractArray{F} where F<:Real
    zi = cm(m(x))
    g = 1 ./ masterarbeit.jacobian4cl(m, cm, x)
    fracs = abs.(f(zi) .- g) .^F(1.5) ./ f(zi)
    return sum(fracs) / size(x,2)
end

In [ ]:
f = double_gauss
ytozmap = IdentityMapping()
dim = 5
dimA = 3;

In [ ]:
batchsize = 16384
N_epochs = 100
bins = 10
learning_rate = 0.01
decay = 0.7
optimizer = Adam
;

In [ ]:
function subnet(dimA::Signed, dimB::Signed, bins::Signed, width=16)
    return Chain(
        Split(
            Chain(
                BatchNorm(dimA),
                Dense(dimA => width, relu),
                Dense(width => width, relu),
                Dense(width => width, relu),
                Dense(width => dimB*(bins+1))  
                ), 
            Chain(
                BatchNorm(dimA),
                Dense(dimA => width, relu),
                Dense(width => width, relu),
                Dense(width => width, relu),
                Dense(width => dimB*bins)
                )
            ) 
        ) |> gpu
end

In [ ]:
model = Flux.f32(Chain(
    CouplingLayer(dim, dimA, bins, subnet),
    masterarbeit.MaskLayer([false, false, true, true, true]),
    CouplingLayer(dim, dimA, bins, subnet),
    masterarbeit.MaskLayer([false, false, true, true, true]),
    CouplingLayer(dim, dimA, bins, subnet),
    masterarbeit.MaskLayer([false, false, true, true, true]),
    CouplingLayer(dim, dimA, bins, subnet),
) |> gpu);

In [ ]:
# first run to compile
xtest = CUDA.rand(dim,batchsize);

In [ ]:
model(xtest);

In [ ]:
f(ytozmap(xtest));

In [ ]:
lossf(model,ytozmap,f,xtest)

In [ ]:
Flux.withgradient(m-> lossf(m,ytozmap,f,xtest), model);

# Training

In [ ]:
losses = Float32[]

In [ ]:
function train_NN2(model::Chain, dim, lossf, losses, ytozmap, f; epochs=300, batchsize=16384, optimizer=Adam, learning_rate=0.0089, decay=0.05, ftype=Float32)
    # losses is an input because this way train_NN can be run multiple times (with different training parameters) and all losses be collected
    opt_state = Flux.setup(optimizer, model)
    @showprogress for epoch in 1:epochs
        data = CUDA.rand(ftype, dim, batchsize)
        val, grads = Flux.withgradient(
        m-> lossf(m,ytozmap,f,data), model
        )

        # Save the loss from the forward pass. (Done outside of gradient.)
        push!(losses, val)

        Flux.update!(opt_state, model, grads[1])
    end
    return losses
end

In [ ]:
xticks = [0.0, 0.25, 0.5, 0.75, 1.0]
yticks = [0.0, 0.25, 0.5, 0.75, 1.0];

In [ ]:
for i in 1:3
    println("Training with learning rate  = $(learning_rate)")
    losses = train_NN(model, dim, lossf, losses, ytozmap, f, epochs=N_epochs, batchsize=batchsize, optimizer=optimizer, learning_rate=learning_rate)
    learning_rate = learning_rate * decay
end;

In [ ]:
losses[end]

In [ ]:
fig = Figure(size=(1500,1000))
ax = Axis(fig[1,1], xlabel="epoch", ylabel="loss", yscale=log10, xlabelsize=50, ylabelsize=50)
lines!(1:length(losses), losses, linewidth=3, color=wblue, label="loss")
n = 10
lines!(n:length(losses), moving_average(losses, n), linewidth=4, color=worange, label="$n epoch \n moving average")
fig[1,2] = Legend(fig, ax)
save("doublegauss5d_nis_loss.png", fig)
fig

In [ ]:
samples = sample_NN(model, ytozmap, dim, 10^7, batchsize);

In [ ]:
xticks = [0.0, 0.25, 0.5, 0.75, 1.0]
yticks = [0.0, 0.25, 0.5, 0.75, 1.0];

In [ ]:
histo = fit(Histogram, (samples[1,:], samples[2,:]), nbins=100)
histo_n = StatsBase.normalize(histo, mode=:pdf)
fig = Figure(size=(1200,1000), figure_padding=40)
ax = Axis(fig[1,1], xlabel=L"x_1", ylabel=L"x_2", xticks=xticks, yticks=yticks, aspect=1, xlabelsize=50, ylabelsize=50)
hm = heatmap!(histo.edges[1], histo.edges[2], histo_n.weights,  colorrange=(0,5), highclip=cgrad(:viridis)[end])
fig[1, 2] = GridLayout(width = 20)
Colorbar(fig[1,3], hm, width=40)
save("doublegauss5d_nis_samples.png", fig)
fig

In [ ]:
function slice_samples(ind1, ind2, name1, name2)
    histo = fit(Histogram, (samples[ind1,:], samples[ind2,:]), nbins=100)
    histo_n = StatsBase.normalize(histo, mode=:pdf)
    fig = Figure(size=(1000,1000), figure_padding=40)
    ax = Axis(fig[1,1], xlabel=latexstring(name1), xticks=xticks, yticks=yticks, aspect=1)
    hidexdecorations!(ax, ticks=false)
    hideydecorations!(ax, ticks=false)
    heatmap!(histo.edges[1], histo.edges[2], histo_n.weights, colorrange=(0,5),)
    save("doublegauss5d_nis_samples_"*(name1)*"-"*(name2)*".png", fig)
    fig
end
slice_samples(1, 2, "x_1", "x_2")

In [ ]:
#=slice_samples(1, 2, "x_1", "x_2")
slice_samples(1, 3, "x_1", "x_3")
slice_samples(1, 4, "x_1", "x_4")
slice_samples(1, 5, "x_1", "x_5")
slice_samples(2, 3, "x_2", "x_3")
slice_samples(2, 4, "x_2", "x_4")
slice_samples(2, 5, "x_2", "x_5")
slice_samples(3, 4, "x_3", "x_4")
slice_samples(3, 5, "x_3", "x_5")
slice_samples(3, 5, "x_3", "x_5")
slice_samples(4, 5, "x_4", "x_5")=#

In [ ]:
fig = Figure(size=(1200,1000), figure_padding=40)
ax = Axis(fig[1,1], xlabel=L"x_1", ylabel=L"x_2", xticks=xticks, yticks=yticks, aspect=1, xlabelsize=50, ylabelsize=50)
xs = LinRange(0.0, 1.0, 1001)
ys = LinRange(0.0, 1.0, 1001)
zs = [f([x;y])[1] for x in xs, y in ys]
heatmap!(xs, ys, zs)
save("doublegauss5d_truth.png", fig)
fig

In [ ]:
wi_vegas = load_object("vegas_weights.jld2");

In [ ]:
function weights4cl(m::Chain, cm::ChannelMapping, f::Function, x::T) where {T <: AbstractArray}
    return masterarbeit.jacobian4cl(m, cm, x) .* f(cm(m(x)))
end

function weights4cl_chunked(m, dim, cm, f, N, batchsize)
    if (N%batchsize != 0) 
        x = CUDA.rand(dim, N%batchsize)
        weights = weights4cl(m, cm, f, x) |> cpu
        inputs = x
        runs = N ÷ batchsize 
    else
        x = CUDA.rand(dim,   batchsize)
        weights = weights4cl(m, cm, f, x) |> cpu
        inputs = x
        runs = N ÷ batchsize - 1
    end
    for i in 1:runs
        x = CUDA.rand(dim, batchsize)
        weights = hcat(weights, weights4cl(m, cm, f, x) |> cpu)
        inputs = hcat(inputs, x)
    end
    return weights, inputs
end

wi_m, x_for_wi = weights4cl_chunked(model, dim, ytozmap, f,  10^7, batchsize);

In [ ]:
wi = wi_m[1,:]
f_over_g = wi
mcint = 0.0088
nis_int = sum(f_over_g) / size(samples,2)
mcerror = sqrt(sum((f_over_g  .- mcint).^2) / (size(samples,2)-1))
println("mc integral = $nis_int")
println("standard deviation = $mcerror")

In [ ]:
w_n = wi_m[1,:] ./ mcint
w_avg = mean(w_n)
w_max = maximum(w_n)
efficiency = w_avg / w_max
println("mean weight = $w_avg")
println("max weight = $w_max")
println("unweighting efficiency = $efficiency")

In [ ]:
wi_vegas_filtered = wi_vegas[wi_vegas .< 5.01];

### Normalization
Integral in 1d = 0.340825

Integral in 2d = 0.340825 / 2.93405

Integral in 5d = 0.340825 / 2.93405^4 = 0.00459899

In [ ]:
fig = Figure(size=(1500,1000))
ax = Axis(fig[1,1], ylabel=L"N", xlabel=L"w", yscale=Makie.pseudolog10, yticks=[0, 10^1, 10^3, 10^5, 10^7], backgroundcolor=:white, xlabelsize=50, ylabelsize=50)
stephist!(wi_vegas, color=wblue, linewidth=3, label="VEGAS", bins=100) 
stephist!(w_n, color=worange, linewidth=3, label="NIS", bins=100) 
fig[1,2] = Legend(fig, ax)
inset_ax = Axis(fig[1, 1], yscale=Makie.pseudolog10, xticks=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0], yticks=[0, 10^1, 10^3, 10^5, 10^7], 
    width=Relative(0.65), height=Relative(0.65), halign=0.92, valign=0.92, backgroundcolor=:white)

translate!(inset_ax.elements[:background], 0, 0, -10)
stephist!(wi_vegas_filtered, color=wblue, linewidth=3, label="VEGAS", bins=2000) 
stephist!(w_n, color=worange, linewidth=3, label="NIS", bins=2000) 
xlims!(0.0, 5.0)
save("doublegauss5d_weights.png", fig)
fig